In [2]:
from coffea import hist
import math
import os
import psutil
import uproot
import awkward as ak
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
import random
plt.style.use(hep.style.CMS)

from matplotlib import colors
POPTS={'norm':colors.LogNorm(vmin=1,vmax=200)}

In [3]:
with uproot.open("data/protonpion_sep7/pion_sep7_ntuple/pion_ntuple.root") as file:
    #print(file["Events"].keys())
    print(file["Events"]["Hcal_RecHit_PE"].array())
    print([file["Events"]["Hcal_RecHit_layer"].array()==1])
    #for attr in file["Events"]

[[7, 2, 2, 1], [0, 93, 1, 21, 7, ... 276, 79, 74, 76, 90, 97, 121, 187, 41, 12, 4]]
[<Array [[False, False, ... False, False]] type='10000 * var * bool'>]


In [4]:
SimParticle_attrs = ['pdgID','trkID','mass','e','kine','px','py','pz','endx','endy','endz','vx','vy','vz']
EcalRecHit_attrs = ['amp','e','t','x','y','z']
EcalInfo_attrs = ['frontmaxSP_e','frontmaxSP_p','backmaxSP_e','backmaxSP_p']
HcalRecHit_attrs = ['layer','strip','section','e','x','y','z','PE']
HcalInfo_attrs = ['sumPE']

branches = {
    "Sim_Particle": SimParticle_attrs,
    "Ecal_RecHit": EcalRecHit_attrs,
    "Ecal": EcalInfo_attrs,
    "Hcal_RecHit": HcalRecHit_attrs,
    "Hcal": HcalInfo_attrs,
    "n": ["Ecal_RecHit", "Hcal_RecHit"]
}

def getData(fnames="", treeName="Events", chunks=False):
    branchlist = []
    for collection, attrs in branches.items():
        branchlist += [collection+"_"+attr for attr in attrs]
    if chunks: ldmx_dict = uproot.iterate(fnames+":"+treeName, branchlist)
    else: ldmx_dict = uproot.lazy(fnames+":"+treeName, branchlist)
    return ldmx_dict

#Repackages ldmx_dict into new dictionary of dictionaries of form
#ldmx_events={Sim_particle: {pdgID:___, trkID:___,...}, Ecal_RecHit: {amp:___, e:___, ...}, ...}
def repackage(ldmx_dict):
    evt_dict={}
    for collection in branches:    
        coll_dict={}
        for attr in branches[collection]:
            bname = "{}_{}".format(collection, attr)
            coll_dict[attr] = ldmx_dict[bname]
        evt_dict[collection] = ak.zip(coll_dict)        
    ldmx_events = ak.zip(evt_dict, depth_limit=1)
    return ldmx_events


def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x


#---------------------------------- Histogram Functions---------------------------------------------------
Bins={"Sim_Particle": 
        {'pdgID':[50, 200, 2240], 'trkID':[50, 0, 300], 'mass':[20, 930, 940], 'e':[50, 500, 3500],
         'kine':[50, 0, 2300], 'px':[50, 0, 1000], 'py':[50, 0, 1000], 'pz':[50, 0, 3000],
         'endx':[50, 0, 500],'endy':[50, 0, 500], 'endz':[50, 0, 2700], 'vx':[50, 0, 500],
         'vy':[50, 0, 500], 'vz':[50, 0, 2500]},
    
      "Ecal_RecHit": 
        {'amp':[50, 0, 6], 'e':[50, 0, 500], 't':[20, 0, 1], 
         'x':[50, 0, 250], 'y':[50, 0, 260], 'z':[50, 200, 700]},
      
      "Ecal": 
        {'frontmaxSP_e':[50, 2930, 2950], 'frontmaxSP_p':[20, 0, 15], 
         'backmaxSP_e':[50, 0, 2600], 'backmaxSP_p':[50, 0, 300]},
      
      "Hcal_RecHit":
        {'layer':[50, 0, 100], 'strip':[31, 0, 62], 'section':[5, 0, 5], 'e':[50, 0, 30],
         'x':[50, 0, 1600], 'y':[50, 0, 1600], 'z':[50, 0, 4000], 'PE':[50, 0, 400]},
      
      "Hcal":
        {'sumPE':[50, 0, 3000]},
      
      "n":
        {'Ecal_RecHit':[20, 0, 90], 'Hcal_RecHit':[20, 0, 65]}
     }

Labels={"Sim_Particle": 
        {'pdgID':'pdgID', 'trkID':'trkID', 'mass':"Mass [MeV]", 'e':"Energy [MeV]",
         'kine':"Kinetic Energy [MeV]", 'px':r"$p_x$ [MeV]", 'py':r"$p_y$ [MeV]", 'pz':r"$p_z$ [MeV]",
         'endx':r"$End_x$ [mm]",'endy':r"$End_y$ [mm]", 'endz':r"$End_z$ [mm]", 'vx':r"$v_x$ [mm]",
         'vy':r"$v_y$ [mm]", 'vz':r"$v_z$ [mm]"},
    
      "Ecal_RecHit": 
        {'amp':"Amplitude [mA]", 'e':"Energy [MeV]", 't':"Time [s]           ", 
         'x': "x [mm]", 'y':"y [mm]", 'z':"z [mm]"},
      
      "Ecal": 
        {'frontmaxSP_e':"Front Scoring Plane Energy [MeV]", 'frontmaxSP_p':"Front Scoring Plane p [MeV]", 
         'backmaxSP_e':"Back Scoring Plane Energy [MeV]", 'backmaxSP_p':"Back Scoring Plane p [MeV]"},
      
      "Hcal_RecHit":
        {'layer':'layer', 'strip':'strip', 'section':'section', 'e':"Energy [MeV]",
         'x':"x [mm]", 'y':"y [mm]", 'z':"z [mm]", 'PE':'Photo Electrons'},
      
      "Hcal":
        {'sumPE':'sumPE'},
        
      "n":
        {'Ecal_RecHit':("Number of Hits in Event", "Ecal_RecHit"), 'Hcal_RecHit':("Number of Hits in Event", "Hcal_RecHit")} 
         }



#Makes Dictionary of hist.Hist objects in same layout as ldmx_events, where each attr. becomes a key.
#AttrDict must be a dictionary. Bin layout is determined by Bins 3d array, which is indexed with BinAttr
#and BinKey. Preferably this will be mapped out so that individual histograms may be additioned to attr.
#keys. and/or separate binning may be provided. Categorical binnings? for now. 
def MakeHists(AttrDict, Bins):
    hists={}
    for coll in AttrDict.keys():
        attrHists={}
        for attr in AttrDict[coll]:
            attrHists[attr]=hist.Hist(coll, 
                    axes=(
                    hist.Cat("particle", "Particle"),
                    hist.Bin("e", attr, *(Bins[coll][attr]))
                    ))
            
        hists[coll]=attrHists
    return hists

#def FillHists

In [5]:
hists=MakeHists(branches, Bins)

sumEStrip=np.zeros(62)
layerMaxPE=np.zeros(100)


#----------------------Sum of Energies
SumEStrip=hist.Hist("Sum of Energy in Strip", axes=(
                            hist.Cat("particle", "Particle"),
                            hist.Bin("e", "Sum of Energy", 62, 0, 51000),
                            hist.Bin("s", "Strip", 62, 0, 62)))

EventSumEStrip=hist.Hist("Events", axes=(
                            hist.Cat("particle", "Particle"),
                            hist.Bin("e", "Sum of Energy", 62, 0.001, 5),
                            hist.Bin("s", "Strip", 62, 0, 62)))

#----------------------Photo Electrons
MaxPELayer=hist.Hist("Max PE of Layer", axes=(
                            hist.Cat("particle", "Particle"),
                            hist.Bin("e", "Max PE", 50, 0, 500),
                            hist.Bin("s", "Layer",100, 0, 100)))

EventMaxPELayer=hist.Hist("Events", axes=(
                            hist.Cat("particle", "Particle"),
                            hist.Bin("e", "Max PE", 50, 0.001, 100),
                            hist.Bin("s", "Layer",40, 0, 40)))

#---------------------Ecal Measurements
MaxSingleCellE=hist.Hist("Highest Single Cell Energy", axes=(
                            hist.Cat("particle", "Particle"),
                            hist.Bin("e", "Highest Single RecHit", 100, 0, 2500)))

EcalTotalE=hist.Hist("Total Ecal Energy", axes=(
                            hist.Cat("particle", "Particle"),
                            hist.Bin("e", "Total Energy Deposited in Ecal", 50, 0, 10000)))

In [6]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)
    
    sim_particle = ldmx_events['Sim_Particle']
    ecal_rechit = ldmx_events['Ecal_RecHit']
    ecal= ldmx_events['Ecal']
    hcal_rechit= ldmx_events['Hcal_RecHit']
    hcal= ldmx_events['Hcal']
    
    #print(ak.to_layout(sim_particle))
    #print(hcal_rechit.PE[hcal_rechit.layer==7])
#--------------------------------------- DATA MASKS --------------------------------------------
    
    #Determines if data file is one for protons or pions
    par=""
    if ak.all(sim_particle.pdgID==2212):
        par="Proton"
    elif ak.all(sim_particle.pdgID==211):
        par="Pion"
#------------------

    #Sum of energy in each strip over all events
    for i in range(1,63):
        sumEStrip[i-1]=ak.sum(hcal_rechit.e[hcal_rechit.strip==i])
     
    #Max PE in each Layer over all events
    for i in range(1,100):  
        layerMaxPE[i-1]=ak.max(hcal_rechit.PE[hcal_rechit.layer==i])
#------------------    
    
    #Sum of energy in each strip per event
    for i in range(1,63):
        eventSums=ak.sum(hcal_rechit.e[hcal_rechit.strip==i], axis=1)
        EventSumEStrip.fill(particle=par, e=eventSums, s=i)
     
    #Max PE in each Layer per event
    for i in range(1,100):  
        eventMax=ak.flatten(ak.max(hcal_rechit.PE[hcal_rechit.layer==i], axis=1), axis=0)
        EventMaxPELayer.fill(particle=par, e=eventMax, s=i)
            
#------------------

    #Ecal total energy Deposited
    ecalTotalE=ak.sum(ecal_rechit.e, axis=-1)
    
    #Highest single cell energy
    maxSingleCellE=ak.max(ecal_rechit.e, axis=-1)
    
    #for i in range(0,len(maxSingleCellE)):
     #   print(maxSingleCellE[i], ecal_rechit.e[i])
    #print(ak.flatten(ak.sum(hcal_rechit.e[hcal_rechit.strip==1], axis=1), axis=0))
    #print(ak.flatten(ak.max(hcal_rechit.PE[hcal_rechit.strip==1], axis=1), axis=0))
#--------------------------------------- HISTOGRAM FILLING -------------------------------------

    for coll in ldmx_events.fields:
        for f in ldmx_events[coll].fields:
            hists[coll][f].fill(particle=par, e= flat(getattr(ldmx_events[coll], f)))
       
    #SumEStrip, MaxPELayer
    layers=np.arange(1,101)
    strips=np.arange(1,63)
    
    SumEStrip.fill(particle=par, e=sumEStrip, s=strips)
    MaxPELayer.fill(particle=par, e=layerMaxPE, s=layers)
    
    #EcalTotalEnergy, MaxSingleCellE
    MaxSingleCellE.fill(particle=par, e=maxSingleCellE)
    EcalTotalE.fill(particle=par, e=ecalTotalE)
    

In [7]:
ldmx_dict_all = getData(chunks=True, fnames="data/protonpion_sep7/*/*.root")

nchunk = 0
for chunk in ldmx_dict_all:
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)

process 1
process 2


In [8]:
import ipywidgets as widgets
%matplotlib widget

'''
i=0
for coll in hists.keys():
    for key in hists[coll]:
        if i%2==0:
            fig, ax=plt.subplots(ncols=2, figsize=(22,10))
        hist.plot1d(hists[coll][key], ax=ax[i%2], clear=False, overlay="particle")
        
        if (type(Labels[coll][key])==tuple):
            ax[i%2].set_xlabel(Labels[coll][key][0])
            ax[i%2].set_ylabel(Labels[coll][key][1])
        else:
            ax[i%2].set_xlabel(Labels[coll][key])
            
        i+=1

'''
widlist=[[coll+"-"+attr for attr in branches[coll]] for coll in branches.keys()]
WidList=np.array([])
for i in widlist:
    WidList=np.append(WidList, i)


fig, axW=plt.subplots(figsize=(10,10))
def plott(attr, compare=True):
    coll=attr[:attr.find("-")]
    key=attr[attr.find("-")+1:]
    hist.plot1d(hists[coll][key], ax=axW, overlay="particle", clear=compare)#, legend_opts={'labels':[attr]})
    
    if (type(Labels[coll][key])==tuple):
        axW.set_xlabel(Labels[coll][key][0])
        axW.set_ylabel(Labels[coll][key][1])
    else:
        axW.set_xlabel(Labels[coll][key])
    #plt.close()
    #display(fig)
   
    
_=widgets.interact(plott, attr=WidList, compare=[("True",0),("False",1)])



fig, ax1=plt.subplots(nrows=2, figsize=(12,20))
hist.plot2d(EventSumEStrip["Proton"].sum("particle"), "s", ax=ax1[0], clear=True)
hist.plot2d(EventSumEStrip["Pion"].sum("particle"), "s", ax=ax1[1], clear=True)
ax1[0].set_xlabel("Hcal Strip")
ax1[0].set_ylabel("Sum of Energy [MeV]")
ax1[1].set_xlabel("Hcal Strip")
ax1[1].set_ylabel("Sum of Energy [MeV]")
ax1[0].set_title("Proton")
ax1[1].set_title("Pion")

fig, ax2=plt.subplots(nrows=2, figsize=(12,20))
hist.plot2d(EventMaxPELayer["Proton"].sum("particle"), "s", ax=ax2[0], clear=True)
hist.plot2d(EventMaxPELayer["Pion"].sum("particle"), "s", ax=ax2[1], clear=True)
ax2[0].set_xlabel("Hcal Layer")
ax2[0].set_ylabel("Max # Photo Electrons")
ax2[1].set_xlabel("Hcal Layer")
ax2[1].set_ylabel("Max # Photo Electrons")
ax2[0].set_title("Proton")
ax2[1].set_title("Pion")

fig, ax3=plt.subplots(nrows=2, figsize=(10,20))
hist.plot1d(MaxSingleCellE, ax=ax3[0], clear=False, overlay="particle")
hist.plot1d(EcalTotalE, ax=ax3[1], clear=False, overlay="particle")
ax3[0].set_xlabel("Highest Single Cell Energy in Ecal [MeV]")
ax3[0].set_ylabel("Events")
ax3[1].set_xlabel("Total Energy in Ecal [MeV] ")
ax3[1].set_ylabel("Events")

"""
fig, ax3=plt.subplots(nrows=2, figsize=(10,20))
hist.plot1d(SumEStrip.sum("s"), ax=ax3[0], clear=False, overlay="particle")
hist.plot1d(MaxPELayer.sum("s"), ax=ax3[1], clear=False, overlay="particle")
ax3[0].set_xlabel("Sum of Energy [MeV]")
ax3[0].set_ylabel("Hcal Strip")
ax3[1].set_xlabel("Max # Photo Electrons")
ax3[1].set_ylabel("Layers")
"""


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='attr', options=('Sim_Particle-pdgID', 'Sim_Particle-trkID', 'Sim_P…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

'\nfig, ax3=plt.subplots(nrows=2, figsize=(10,20))\nhist.plot1d(SumEStrip.sum("s"), ax=ax3[0], clear=False, overlay="particle")\nhist.plot1d(MaxPELayer.sum("s"), ax=ax3[1], clear=False, overlay="particle")\nax3[0].set_xlabel("Sum of Energy [MeV]")\nax3[0].set_ylabel("Hcal Strip")\nax3[1].set_xlabel("Max # Photo Electrons")\nax3[1].set_ylabel("Layers")\n'